In [1]:
from btgp.data_utils import load_dataset
from btgp.gp import ONGP
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

import time

In [5]:
path_start = "../data/uci/"
test_rmses = []
test_nlls = []
train_times = []
predict_times = []
SEEDS = [0,1,2]
torch.manual_seed(0)
for seed in SEEDS:
    train_x, train_y, val_x, val_y, test_x, test_y = load_dataset(
        path_start + "elevators/elevators.mat",
        random_split=seed,
    )
    
    test_y = test_y.to(device=device)
    gp = ONGP(train_x.to(dtype=torch.float64, device=device),
              train_y.to(dtype=torch.float64, device=device),
              test_x.to(dtype=torch.float64, device=device), lambd=1e-4, precision=16)
    gp.random_bit_order()
    gp.process()
    print("Train nll:", gp.nll)
    print("Test nll:", gp.calculate_test_nll(test_y.view(-1,1)))
    print("Test rmse:", gp.rmse(test_y.view(-1,1)))
    print()
    print("Training the weight vector and the bit order:")
    train_time = time.time()
    gp.update_weights_slsqp(update_order=True, verbose=False)
    train_times.append(time.time() - train_time)
    gp.save(path="")
    predict_time = time.time()
    test_nll = gp.calculate_test_nll(test_y.view(-1,1))
    predict_times.append(time.time() - predict_time)
    test_nlls.append(test_nll)
    test_rmse = gp.rmse(test_y.view(-1,1))
    test_rmses.append(test_rmse)
    print("Test nll:", test_nll)
    print("Test rmse:", test_rmse)

dataset_path: ../data/uci/elevators/elevators.mat
data.shape: torch.Size([16599, 19])
train_x.shape: torch.Size([10623, 18])
full kernel size: 0.4 GB
train_x: min 0.000, max 1.000, mean 0.536, std 0.252
val_x: min -0.077, max 1.029, mean 0.536, std 0.253
test_x: min -0.053, max 1.087, mean 0.534, std 0.252
train_y: min -2.193, max 5.220, mean 0.000, std 1.000
val_y: min -2.193, max 5.012, mean -0.043, std 1.003
test_y: min -2.193, max 4.559, mean 0.015, std 0.999

Train nll: tensor(1.2630, device='cuda:1', dtype=torch.float64)
Test nll: tensor(1.2122, device='cuda:1', dtype=torch.float64)
Test rmse: tensor(0.7872, device='cuda:1', dtype=torch.float64)

Training the weight vector and the bit order:
Test nll: tensor(0.9272, device='cuda:1', dtype=torch.float64)
Test rmse: tensor(0.6235, device='cuda:1', dtype=torch.float64)
dataset_path: ../data/uci/elevators/elevators.mat
data.shape: torch.Size([16599, 19])
train_x.shape: torch.Size([10623, 18])
full kernel size: 0.4 GB
train_x: min 0.0

## test SLSQP

In [8]:
test_nlls

[tensor(0.9272, device='cuda:1', dtype=torch.float64),
 tensor(1.0056, device='cuda:1', dtype=torch.float64),
 tensor(0.8316, device='cuda:1', dtype=torch.float64)]

In [9]:
test_rmses

[tensor(0.6235, device='cuda:1', dtype=torch.float64),
 tensor(0.6423, device='cuda:1', dtype=torch.float64),
 tensor(0.5310, device='cuda:1', dtype=torch.float64)]